In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from statsmodels.stats.proportion import proportions_ztest
import os
import sys

In [ ]:
DATA_PATH = os.path.join('../data', 'MachineLearningRating_v3.txt')


def load_data(filepath, delimiter='|'):
    """
    Loads the insurance claims data from a text file.
    Handles potential initial loading errors and allows specifying a delimiter.

    Args:
        filepath (str): The path to the data file (.txt).
        delimiter (str): The character used to separate values in the file (e.g., ',', '\t', ';').
                         Defaults to pipe.
    """
    try:
        # pd.read_csv can read .txt files if the delimiter is correctly specified
        df = pd.read_csv(
            filepath,
            delimiter=delimiter,
            low_memory=False  # Add this option
        )
        print(f"Data loaded successfully from {filepath} with delimiter '{delimiter}'. Shape: {df.shape}")
        return df
    except FileNotFoundError:
        print(f"Error: File not found at {filepath}. Please ensure the data TXT is in the 'data' directory and named 'insurance_claims.txt'.")
        return None
    except Exception as e:
        print(f"An error occurred while loading data: {e}\n"
              "Please check if the delimiter is correct and the file format is consistent.")
        return None
    
df=load_data(DATA_PATH)


Data loaded successfully from ../data\MachineLearningRating_v3.txt with delimiter '|'. Shape: (1000098, 52)


,UnderwrittenCoverID,PolicyID,TransactionMonth,IsVATRegistered,Citizenship,LegalType,Title,Language,Bank,AccountType,...,ExcessSelected,CoverCategory,CoverType,CoverGroup,Section,Product,StatutoryClass,StatutoryRiskType,TotalPremium,TotalClaims
0,145249,12827,2015-03-01 00:00:00,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Mobility - Windscreen,Windscreen,Windscreen,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,21.929825,0.0
1,145249,12827,2015-05-01 00:00:00,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Mobility - Windscreen,Windscreen,Windscreen,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,21.929825,0.0
2,145249,12827,2015-07-01 00:00:00,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Mobility - Windscreen,Windscreen,Windscreen,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,0.000000,0.0
3,145255,12827,2015-05-01 00:00:00,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Mobility - Metered Taxis - R2000,Own damage,Own Damage,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,512.848070,0.0
4,145255,12827,2015-07-01 00:00:00,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Mobility - Metered Taxis - R2000,Own damage,Own Damage,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,0.000000,0.0


In [17]:
print(df.columns)

Index(['UnderwrittenCoverID', 'PolicyID', 'TransactionMonth',
       'IsVATRegistered', 'Citizenship', 'LegalType', 'Title', 'Language',
       'Bank', 'AccountType', 'MaritalStatus', 'Gender', 'Country', 'Province',
       'PostalCode', 'MainCrestaZone', 'SubCrestaZone', 'ItemType', 'mmcode',
       'VehicleType', 'RegistrationYear', 'make', 'Model', 'Cylinders',
       'cubiccapacity', 'kilowatts', 'bodytype', 'NumberOfDoors',
       'VehicleIntroDate', 'CustomValueEstimate', 'AlarmImmobiliser',
       'TrackingDevice', 'CapitalOutstanding', 'NewVehicle', 'WrittenOff',
       'Rebuilt', 'Converted', 'CrossBorder', 'NumberOfVehiclesInFleet',
       'SumInsured', 'TermFrequency', 'CalculatedPremiumPerTerm',
       'ExcessSelected', 'CoverCategory', 'CoverType', 'CoverGroup', 'Section',
       'Product', 'StatutoryClass', 'StatutoryRiskType', 'TotalPremium',
       'TotalClaims'],
      dtype='object')


In [18]:
def preprocess_data_for_analysis(df):
    """
    Performs preprocessing steps: type conversions, NaN handling, and feature engineering.
    This version is more robust for hypothesis testing and modeling.
    """
    df_processed = df.copy()

    # Convert 'TransactionMonth' to datetime objects
    if 'TransactionMonth' in df_processed.columns:
        df_processed['TransactionDate'] = pd.to_datetime(df_processed['TransactionMonth'], errors='coerce')
        df_processed['TransactionYearMonth'] = df_processed['TransactionDate'].dt.to_period('M')
        # Handle TransactionMonth which might be YYMM format
        df_processed['TransactionMonth_dt'] = df_processed['TransactionDate'].astype(str).apply(
            lambda x: pd.to_datetime(f'20{x[:2]}-{x[2:]}-01', errors='coerce') if len(x) == 4 else np.nan
        )
    else:
        print("'TransactionDate' column not found. Skipping datetime conversion.")

    # Ensure numerical columns are numeric, coercing errors to NaN
    numerical_cols = ['TotalPremium', 'TotalClaims', 'CustomValueEstimate', 'Cylinders',
                      'cubiccapacity', 'kilowatts', 'NumberOfDoors', 'CapitalOutstanding',
                      'NumberOfVehiclesInFleet', 'SumInsured', 'CalculatedPremiumPerTerm',
                      'ExcessSelected', 'RegistrationYear']
    for col in numerical_cols:
        if col in df_processed.columns:
            df_processed[col] = pd.to_numeric(df_processed[col], errors='coerce')
        else:
            print(f"Warning: Numerical column '{col}' not found in DataFrame. Skipping processing for this column.")


    # --- IMPORTANT: Check for critical columns before creating 'HadClaim', 'LossRatio', 'Margin' ---
    missing_critical_cols = []
    if 'TotalPremium' not in df_processed.columns:
        missing_critical_cols.append('TotalPremium')
    if 'TotalClaims' not in df_processed.columns:
        missing_critical_cols.append('TotalClaims')

    if missing_critical_cols:
        print(f"Error: Missing critical columns for risk/profit analysis: {', '.join(missing_critical_cols)}. Cannot proceed with 'HadClaim', 'LossRatio', 'Margin' calculation.")
        # Return df_processed even if critical columns are missing, but subsequent calculations will fail
        # This allows the script to continue for debugging purposes, but the tests will skip.
        return df_processed

    # Create 'HadClaim' binary variable: 1 if TotalClaims > 0, else 0
    df_processed['HadClaim'] = (df_processed['TotalClaims'] > 0).astype(int)

    # Calculate Loss Ratio (TotalClaims / TotalPremium)
    # Handle cases where TotalPremium is 0 or NaN to avoid division errors
    df_processed['LossRatio'] = np.where(
        (df_processed['TotalPremium'].notnull()) & (df_processed['TotalPremium'] > 0),
        df_processed['TotalClaims'] / df_processed['TotalPremium'],
        0 # Assign 0 if premium is 0 or null, or handle as NaN if preferred
    )

    # Calculate Margin (TotalPremium - TotalClaims)
    df_processed['Margin'] = df_processed['TotalPremium'] - df_processed['TotalClaims']
    print("Checked: 'Margin' column successfully created.")

    # Convert object columns that should be categorical to 'category' dtype
    categorical_cols = ['IsVATRegistered', 'Citizenship', 'LegalType', 'Title', 'Language',
                        'Bank', 'AccountType', 'MaritalStatus', 'Gender', 'Country', 'Province',
                        'ItemType', 'VehicleType', 'make', 'Model', 'bodytype', 'AlarmImmobiliser',
                        'TrackingDevice', 'NewVehicle', 'WrittenOff', 'Rebuilt', 'Converted',
                        'CrossBorder', 'CoverCategory', 'CoverType', 'CoverGroup', 'Section',
                        'Product', 'StatutoryClass', 'StatutoryRiskType', 'MainCrestaZone', 'SubCrestaZone']
    for col in categorical_cols:
        if col in df_processed.columns:
            df_processed[col] = df_processed[col].astype('category')
        else:
            print(f"Warning: Categorical column '{col}' not found in DataFrame. Skipping processing for this column.")


    # Convert PostalCode to string, fill NaNs if any
    if 'PostalCode' in df_processed.columns:
        df_processed['PostalCode'] = df_processed['PostalCode'].astype(str).fillna('UNKNOWN')
    else:
        print("Warning: 'PostalCode' column not found in DataFrame. Skipping processing for this column.")

    print("Data preprocessing for analysis complete. Added 'HadClaim', 'LossRatio', 'Margin'.")
    return df_processed


prep_df=preprocess_data_for_analysis(df)
prep_df.head()

Checked: 'Margin' column successfully created.
Data preprocessing for analysis complete. Added 'HadClaim', 'LossRatio', 'Margin'.


,UnderwrittenCoverID,PolicyID,TransactionMonth,IsVATRegistered,Citizenship,LegalType,Title,Language,Bank,AccountType,...,StatutoryClass,StatutoryRiskType,TotalPremium,TotalClaims,TransactionDate,TransactionYearMonth,TransactionMonth_dt,HadClaim,LossRatio,Margin
0,145249,12827,2015-03-01 00:00:00,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Commercial,IFRS Constant,21.929825,0.0,2015-03-01,2015-03,NaN,0,0.0,21.929825
1,145249,12827,2015-05-01 00:00:00,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Commercial,IFRS Constant,21.929825,0.0,2015-05-01,2015-05,NaN,0,0.0,21.929825
2,145249,12827,2015-07-01 00:00:00,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Commercial,IFRS Constant,0.000000,0.0,2015-07-01,2015-07,NaN,0,0.0,0.000000
3,145255,12827,2015-05-01 00:00:00,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Commercial,IFRS Constant,512.848070,0.0,2015-05-01,2015-05,NaN,0,0.0,512.848070
4,145255,12827,2015-07-01 00:00:00,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Commercial,IFRS Constant,0.000000,0.0,2015-07-01,2015-07,NaN,0,0.0,0.000000


In [19]:
# --- Metric Calculation Functions ---

def calculate_claim_frequency(df_segment):
    """Calculates claim frequency for a given DataFrame segment."""
    if df_segment.empty:
        return 0.0
    return df_segment['HadClaim'].mean() # Mean of 0s and 1s gives the proportion

def calculate_claim_severity(df_segment):
    """
    Calculates claim severity for a given DataFrame segment (only for policies with claims).
    Returns NaN if no claims or segment is empty.
    """
    claims_df = df_segment[df_segment['TotalClaims'] > 0]
    if claims_df.empty:
        return np.nan
    return claims_df['TotalClaims'].mean()

def calculate_average_margin(df_segment):
    """Calculates the average margin for a given DataFrame segment."""
    if df_segment.empty:
        return np.nan
    return df_segment['Margin'].mean()

# --- Statistical Testing Functions ---

def perform_chi_squared_test(group1_counts, group2_counts, alpha=0.05):
    """
    Performs a Chi-squared test for independence between two groups' claim frequencies.
    Args:
        group1_counts (tuple): (count_claims_group1, total_policies_group1)
        group2_counts (tuple): (count_claims_group2, total_policies_group2)
        alpha (float): Significance level.
    Returns:
        tuple: (statistic, p_value, interpretation_string)
    """
    # Create an observed frequency table for the chi-squared test
    # Row 0: Claim, Row 1: No Claim
    # Col 0: Group 1, Col 1: Group 2
    observed = np.array([
        [group1_counts[0], group2_counts[0]],
        [group1_counts[1] - group1_counts[0], group2_counts[1] - group2_counts[0]]
    ])

    if np.any(observed < 5) and (np.min(observed.sum(axis=0)) < 30): # Check for small expected frequencies or small samples
        # Fallback to Fisher's exact test if Chi-squared assumptions are violated
        # However, for typical large datasets, chi-squared is generally robust.
        # For simplicity, we proceed with chi-squared but acknowledge this edge case.
        pass

    chi2, p_value, _, _ = stats.chi2_contingency(observed)

    interpretation = f"Chi-squared statistic: {chi2:.3f}, P-value: {p_value:.3f}. "
    if p_value < alpha:
        interpretation += f"Reject H₀: There is a statistically significant difference in claim frequency (p < {alpha})."
    else:
        interpretation += f"Fail to reject H₀: No statistically significant difference in claim frequency (p >= {alpha})."
    return chi2, p_value, interpretation
def perform_ttest(group1_data, group2_data, metric_name, alpha=0.05):
    """
    Performs an independent t-test on two groups' numerical data (e.g., Claim Severity, Margin).
    Args:
        group1_data (pd.Series): Data for the first group.
        group2_data (pd.Series): Data for the second group.
        metric_name (str): Name of the metric being tested (for reporting).
        alpha (float): Significance level.
    Returns:
        tuple: (statistic, p_value, interpretation_string)
    """
    # Remove NaNs before t-test
    group1_data = group1_data.dropna()
    group2_data = group2_data.dropna()

    if len(group1_data) < 2 or len(group2_data) < 2: # Need at least 2 samples for a t-test
        return np.nan, np.nan, f"Insufficient data for t-test on {metric_name}. Group sizes: {len(group1_data)}, {len(group2_data)}."

    # Perform independent t-test (assuming unequal variances for robustness: Welch's t-test)
    t_statistic, p_value = stats.ttest_ind(group1_data, group2_data, equal_var=False)

    interpretation = f"T-statistic: {t_statistic:.3f}, P-value: {p_value:.3f}. "
    if p_value < alpha:
        interpretation += f"Reject H₀: There is a statistically significant difference in {metric_name} (p < {alpha})."
    else:
        interpretation += f"Fail to reject H₀: No statistically significant difference in {metric_name} (p >= {alpha})."
    return t_statistic, p_value, interpretation


In [21]:
# --- Hypothesis Testing Execution ---

def run_hypothesis_tests(df, alpha=0.05):
    """
    Executes all defined hypothesis tests and reports findings.
    """
    print("\n" + "="*50)
    print("      Starting A/B Hypothesis Testing      ")
    print("="*50 + "\n")

    # --- H₀ 1: No risk differences across provinces ---
    print("\n--- Hypothesis 1: No risk differences across provinces ---")
    # For provinces, we need to compare multiple groups. A simple A/B test would involve picking
    # two provinces. For a more comprehensive analysis (beyond simple A/B), ANOVA might be used for
    # means, and Chi-squared for proportions across multiple groups.
    # Here, we'll pick the two provinces with the highest and lowest loss ratios from EDA.
    # Or, we can compare a high-risk province against the rest.
    
    # First, calculate overall metrics per province to identify candidates for comparison
    province_metrics = df.groupby('Province').agg(TotalPolicies=('PolicyID', 'nunique'),ClaimsCount=('HadClaim', 'sum'),TotalClaimsAmount=('TotalClaims', 'sum'),TotalPremiumAmount=('TotalPremium', 'sum')).reset_index()
    province_metrics['ClaimFrequency'] = province_metrics['ClaimsCount'] / province_metrics['TotalPolicies']
    province_metrics['ClaimSeverity'] = province_metrics.apply(lambda row: row['TotalClaimsAmount'] / row['ClaimsCount'] if row['ClaimsCount'] > 0 else np.nan,axis=1)
    province_metrics['AvgMargin'] = (province_metrics['TotalPremiumAmount'] - province_metrics['TotalClaimsAmount']) / province_metrics['TotalPolicies']

    # Filter out provinces with very few policies if they skew results
    province_metrics = province_metrics[province_metrics['TotalPolicies'] > 100].sort_values('ClaimFrequency', ascending=False) # Example filter

    if not province_metrics.empty:
        # Select a "high-risk" and "low-risk" province based on claim frequency
        province_high_freq = province_metrics.iloc[0]['Province'] if not province_metrics.empty else None
        province_low_freq = province_metrics.iloc[-1]['Province'] if len(province_metrics) > 1 else None

        if province_high_freq and province_low_freq and province_high_freq != province_low_freq:
            print(f"Comparing {province_high_freq} (High Claim Freq) vs. {province_low_freq} (Low Claim Freq)")

            df_prov_high = df[df['Province'] == province_high_freq].dropna(subset=['HadClaim', 'TotalClaims', 'TotalPremium'])
            df_prov_low = df[df['Province'] == province_low_freq].dropna(subset=['HadClaim', 'TotalClaims', 'TotalPremium'])

            if df_prov_high.empty or df_prov_low.empty:
                print("Not enough data for province comparison after dropping NaNs.")
            else:
                # Test Claim Frequency
                count_high_freq_claim = df_prov_high['HadClaim'].sum()
                total_high_freq_policies = len(df_prov_high)
                count_low_freq_claim = df_prov_low['HadClaim'].sum()
                total_low_freq_policies = len(df_prov_low)

                print(f"Claim Frequency for {province_high_freq}: {calculate_claim_frequency(df_prov_high):.4f}")
                print(f"Claim Frequency for {province_low_freq}: {calculate_claim_frequency(df_prov_low):.4f}")

                chi2_stat, p_val_freq, interpretation_freq = perform_chi_squared_test(
                    (count_high_freq_claim, total_high_freq_policies),
                    (count_low_freq_claim, total_low_freq_policies), alpha
                )
                print(f"  Claim Frequency (Proportions Z-test - internally uses chi-squared logic for large samples): {interpretation_freq}")
                if p_val_freq < alpha:
                    print(f"  Business Insight: Risk (Claim Frequency) varies significantly by province. {province_high_freq} likely requires higher premiums or targeted risk mitigation compared to {province_low_freq}.")

                # Test Claim Severity (only for policies with claims)
                severity_high = df_prov_high[df_prov_high['HadClaim'] == 1]['TotalClaims']
                severity_low = df_prov_low[df_prov_low['HadClaim'] == 1]['TotalClaims']
                print(f"Claim Severity for {province_high_freq}: {calculate_claim_severity(df_prov_high):.2f}")
                print(f"Claim Severity for {province_low_freq}: {calculate_claim_severity(df_prov_low):.2f}")

                t_stat_severity, p_val_severity, interpretation_severity = perform_ttest(severity_high, severity_low, "Claim Severity", alpha)
                print(f"  Claim Severity (T-test): {interpretation_severity}")
                if p_val_severity < alpha and not np.isnan(p_val_severity):
                    print(f"  Business Insight: Claim severity also differs significantly. Focus on understanding factors driving higher claim costs in {province_high_freq}.")

                # Test Average Margin
                margin_high = df_prov_high['Margin']
                margin_low = df_prov_low['Margin']
                print(f"Average Margin for {province_high_freq}: {calculate_average_margin(df_prov_high):.2f}")
                print(f"Average Margin for {province_low_freq}: {calculate_average_margin(df_prov_low):.2f}")

                t_stat_margin, p_val_margin, interpretation_margin = perform_ttest(margin_high, margin_low, "Average Margin", alpha)
                print(f"  Average Margin (T-test): {interpretation_margin}")
                if p_val_margin < alpha and not np.isnan(p_val_margin):
                    print(f"  Business Insight: The average profit margin varies significantly between provinces, indicating a need for region-specific pricing adjustments.")
        else:
            print("Not enough distinct provinces with sufficient data to perform meaningful A/B comparison based on initial frequency sorting.")
    else:
        print("No provinces with sufficient data to perform comparison.")


    # --- H₀ 2: No risk differences between zip codes ---
    print("\n--- Hypothesis 2: No risk differences between zip codes ---")
    # Due to the large number of zip codes, we cannot compare all pairs.
    # Instead, we'll pick a few prominent zip codes (e.g., top N by policies)
    # and compare their risk metrics against the overall average, or pick a high-risk vs low-risk zip.
    # For demonstration, let's pick two zip codes with high number of policies and compare them.
    
    # Calculate metrics for each postal code
    postal_code_metrics = df.groupby('PostalCode').agg(TotalPolicies=('PolicyID', 'nunique'),ClaimsCount=('HadClaim', 'sum'),TotalClaimsAmount=('TotalClaims', 'sum'),TotalPremiumAmount=('TotalPremium', 'sum')).reset_index()
    postal_code_metrics['ClaimFrequency'] = postal_code_metrics['ClaimsCount'] / postal_code_metrics['TotalPolicies']
    postal_code_metrics['ClaimSeverity'] = postal_code_metrics.apply(lambda row: row['TotalClaimsAmount'] / row['ClaimsCount'] if row['ClaimsCount'] > 0 else np.nan,axis=1)
    postal_code_metrics['AvgMargin'] = (postal_code_metrics['TotalPremiumAmount'] - postal_code_metrics['TotalClaimsAmount']) / postal_code_metrics['TotalPolicies']

    # Filter out zip codes with very few policies and sort by claim frequency for picking candidates
    postal_code_metrics = postal_code_metrics[postal_code_metrics['TotalPolicies'] > 50].sort_values('ClaimFrequency', ascending=False) # Example filter

    if len(postal_code_metrics) >= 2:
        zip_high_freq = postal_code_metrics.iloc[0]['PostalCode']
        zip_low_freq = postal_code_metrics.iloc[-1]['PostalCode']

        print(f"Comparing Zip Code {zip_high_freq} (High Claim Freq) vs. Zip Code {zip_low_freq} (Low Claim Freq)")

        df_zip_high = df[df['PostalCode'] == zip_high_freq].dropna(subset=['HadClaim', 'TotalClaims', 'TotalPremium'])
        df_zip_low = df[df['PostalCode'] == zip_low_freq].dropna(subset=['HadClaim', 'TotalClaims', 'TotalPremium'])

        if df_zip_high.empty or df_zip_low.empty:
            print("Not enough data for zip code comparison after dropping NaNs.")
        else:
            # Test Claim Frequency
            count_high_zip_claim = df_zip_high['HadClaim'].sum()
            total_high_zip_policies = len(df_zip_high)
            count_low_zip_claim = df_zip_low['HadClaim'].sum()
            total_low_zip_policies = len(df_zip_low)

            print(f"Claim Frequency for Zip {zip_high_freq}: {calculate_claim_frequency(df_zip_high):.4f}")
            print(f"Claim Frequency for Zip {zip_low_freq}: {calculate_claim_frequency(df_zip_low):.4f}")

            chi2_zip_freq, p_val_zip_freq, interpretation_zip_freq = perform_chi_squared_test((count_high_zip_claim, total_high_zip_policies),(count_low_zip_claim, total_low_zip_policies), alpha)
            print(f"  Claim Frequency (Proportions Z-test): {interpretation_zip_freq}")
            if p_val_zip_freq < alpha:
                print(f"  Business Insight: Risk (Claim Frequency) varies significantly by zip code. Consider localized premium adjustments or marketing efforts for high-risk zip codes like {zip_high_freq}.")

            # Test Claim Severity
            severity_zip_high = df_zip_high[df_zip_high['HadClaim'] == 1]['TotalClaims']
            severity_zip_low = df_zip_low[df_zip_low['HadClaim'] == 1]['TotalClaims']
            print(f"Claim Severity for Zip {zip_high_freq}: {calculate_claim_severity(df_zip_high):.2f}")
            print(f"Claim Severity for Zip {zip_low_freq}: {calculate_claim_severity(df_zip_low):.2f}")

            t_stat_zip_severity, p_val_zip_severity, interpretation_zip_severity = perform_ttest(
                severity_zip_high, severity_zip_low, "Claim Severity", alpha
            )
            print(f"  Claim Severity (T-test): {interpretation_zip_severity}")
            if p_val_zip_severity < alpha and not np.isnan(p_val_zip_severity):
                print(f"  Business Insight: Claim severity also differs significantly between zip codes, indicating varying cost implications for claims in different areas.")
    else:
        print("Not enough distinct zip codes with sufficient data to perform meaningful A/B comparison.")


    # --- H₀ 3: No significant margin (profit) difference between zip codes ---
    print("\n--- Hypothesis 3: No significant margin (profit) difference between zip codes ---")
    if len(postal_code_metrics) >= 2:
        # Re-using the same high/low frequency zips for margin comparison for consistency
        # Or, you could sort by AvgMargin to pick highest/lowest margin zips
        zip_high_margin = postal_code_metrics.sort_values('AvgMargin', ascending=False).iloc[0]['PostalCode']
        zip_low_margin = postal_code_metrics.sort_values('AvgMargin', ascending=True).iloc[0]['PostalCode']

        print(f"Comparing Zip Code {zip_high_margin} (High Margin) vs. Zip Code {zip_low_margin} (Low Margin)")

        df_zip_high_margin = df[df['PostalCode'] == zip_high_margin].dropna(subset=['Margin'])
        df_zip_low_margin = df[df['PostalCode'] == zip_low_margin].dropna(subset=['Margin'])

        if df_zip_high_margin.empty or df_zip_low_margin.empty:
            print("Not enough data for zip code margin comparison after dropping NaNs.")
        else:
            margin_high_zip_data = df_zip_high_margin['Margin']
            margin_low_zip_data = df_zip_low_margin['Margin']

            print(f"Average Margin for Zip {zip_high_margin}: {calculate_average_margin(df_zip_high_margin):.2f}")
            print(f"Average Margin for Zip {zip_low_margin}: {calculate_average_margin(df_zip_low_margin):.2f}")

            t_stat_zip_margin, p_val_zip_margin, interpretation_zip_margin = perform_ttest(
                margin_high_zip_data, margin_low_zip_data, "Average Margin", alpha
            )
            print(f"  Average Margin (T-test): {interpretation_zip_margin}")
            if p_val_zip_margin < alpha and not np.isnan(p_val_zip_margin):
                print(f"  Business Insight: Profitability (margin) differs significantly between zip codes. Marketing efforts should target high-margin areas like {zip_high_margin}, and pricing adjustments may be needed for low-margin areas like {zip_low_margin}.")
    else:
        print("Not enough distinct zip codes with sufficient data for margin comparison.")


    # --- H₀ 4: No significant risk difference between Women and Men ---
    print("\n--- Hypothesis 4: No significant risk difference between Women and Men ---")
    df_gender = df[df['Gender'].isin(['Male', 'Female'])].dropna(subset=['HadClaim', 'TotalClaims', 'TotalPremium'])

    if df_gender.empty:
        print("No valid 'Male' or 'Female' data available for gender comparison after dropping NaNs.")
    else:
        df_men = df_gender[df_gender['Gender'] == 'Male']
        df_women = df_gender[df_gender['Gender'] == 'Female']

        if df_men.empty or df_women.empty:
            print("Not enough data for both 'Male' and 'Female' groups after filtering.")
        else:
            # Test Claim Frequency
            count_men_claim = df_men['HadClaim'].sum()
            total_men_policies = len(df_men)
            count_women_claim = df_women['HadClaim'].sum()
            total_women_policies = len(df_women)

            print(f"Claim Frequency for Men: {calculate_claim_frequency(df_men):.4f}")
            print(f"Claim Frequency for Women: {calculate_claim_frequency(df_women):.4f}")

            chi2_gender_freq, p_val_gender_freq, interpretation_gender_freq = perform_chi_squared_test(
                (count_men_claim, total_men_policies),
                (count_women_claim, total_women_policies), alpha
            )
            print(f"  Claim Frequency (Proportions Z-test): {interpretation_gender_freq}")
            if p_val_gender_freq < alpha:
                gender_with_higher_freq = "Men" if calculate_claim_frequency(df_men) > calculate_claim_frequency(df_women) else "Women"
                print(f"  Business Insight: There is a statistically significant difference in claim frequency between genders. {gender_with_higher_freq} show a different claim likelihood.")

            # Test Claim Severity (only for policies with claims)
            severity_men = df_men[df_men['HadClaim'] == 1]['TotalClaims']
            severity_women = df_women[df_women['HadClaim'] == 1]['TotalClaims']

            print(f"Claim Severity for Men: {calculate_claim_severity(df_men):.2f}")
            print(f"Claim Severity for Women: {calculate_claim_severity(df_women):.2f}")

            t_stat_gender_severity, p_val_gender_severity, interpretation_gender_severity = perform_ttest(
                severity_men, severity_women, "Claim Severity", alpha
            )
            print(f"  Claim Severity (T-test): {interpretation_gender_severity}")
            if p_val_gender_severity < alpha and not np.isnan(p_val_gender_severity):
                gender_with_higher_severity = "Men" if calculate_claim_severity(df_men) > calculate_claim_severity(df_women) else "Women"
                print(f"  Business Insight: Claim severity also differs significantly by gender, implying varying financial impact for claims based on gender.")

            # Test Average Margin
            margin_men = df_men['Margin']
            margin_women = df_women['Margin']

            print(f"Average Margin for Men: {calculate_average_margin(df_men):.2f}")
            print(f"Average Margin for Women: {calculate_average_margin(df_women):.2f}")

            t_stat_gender_margin, p_val_gender_margin, interpretation_gender_margin = perform_ttest(
                margin_men, margin_women, "Average Margin", alpha
            )
            print(f"  Average Margin (T-test): {interpretation_gender_margin}")
            if p_val_gender_margin < alpha and not np.isnan(p_val_gender_margin):
                gender_with_higher_margin = "Men" if calculate_average_margin(df_men) > calculate_average_margin(df_women) else "Women"
                print(f"  Business Insight: The average margin varies significantly between genders, suggesting gender-specific adjustments to premium or marketing could optimize profitability.")
    print("\n" + "="*50)
    print("      Hypothesis Testing Completed      ")
    print("="*50 + "\n")
    
prep_df.head()  # Display the first few rows of the preprocessed DataFrame
run_hypothesis_tests(prep_df, alpha=0.05)  # Execute the hypothesis tests



      Starting A/B Hypothesis Testing      


--- Hypothesis 1: No risk differences across provinces ---


C:\Users\Perserverence\AppData\Local\Temp\ipykernel_26336\1143745562.py:20: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  province_metrics = df.groupby('Province').agg(TotalPolicies=('PolicyID', 'nunique'),ClaimsCount=('HadClaim', 'sum'),TotalClaimsAmount=('TotalClaims', 'sum'),TotalPremiumAmount=('TotalPremium', 'sum')).reset_index()


Comparing Gauteng (High Claim Freq) vs. Eastern Cape (Low Claim Freq)
Claim Frequency for Gauteng: 0.0034
Claim Frequency for Eastern Cape: 0.0016
  Claim Frequency (Proportions Z-test - internally uses chi-squared logic for large samples): Chi-squared statistic: 24.969, P-value: 0.000. Reject H₀: There is a statistically significant difference in claim frequency (p < 0.05).
  Business Insight: Risk (Claim Frequency) varies significantly by province. Gauteng likely requires higher premiums or targeted risk mitigation compared to Eastern Cape.
Claim Severity for Gauteng: 22243.88
Claim Severity for Eastern Cape: 27128.53
  Claim Severity (T-test): T-statistic: -0.665, P-value: 0.509. Fail to reject H₀: No statistically significant difference in Claim Severity (p >= 0.05).
Average Margin for Gauteng: -13.56
Average Margin for Eastern Cape: 25.83
  Average Margin (T-test): T-statistic: -2.817, P-value: 0.005. Reject H₀: There is a statistically significant difference in Average Margin (p 